In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
train = pd.DataFrame()
for name in os.listdir('proc_data/total')[:-1]:
    ins_data = pd.read_csv(f'proc_data/total/{name}', parse_dates=['date'])
    train = pd.concat([train, ins_data], axis=0).reset_index(drop=True)

In [3]:
train = train.iloc[:, 1:]
train.sample(5)

,Np,Tp,Vp,B_gsm_x,B_gsm_y,B_gsm_z,Bmag,target
6732,-409.841583,18962.867262,-77.158750,0.964905,-2.341024,-0.515571,3.150125,2
21876,-9999.900000,-9999.900000,-12.345119,-2.374899,3.874905,0.663250,6.303881,2
21248,-411.506586,146150.181657,353.107870,-2.791509,1.920053,0.093462,4.817467,4
19297,-296.087327,19912.730357,180.061369,-0.986940,4.884911,1.069327,5.492512,3
17547,-473.612762,419278.455952,378.029107,-5.367619,1.837923,3.041220,8.143185,4


In [4]:
test = pd.read_csv(f"proc_data/total/{os.listdir('proc_data/total')[-1]}", parse_dates=['date'])

In [5]:
test = test.iloc[:, 1:]
test.sample(5)

,Np,Tp,Vp,B_gsm_x,B_gsm_y,B_gsm_z,Bmag,target
2491,-765.659107,28028.294083,-41.878166,-1.764580,-1.867432,3.199320,4.288426,1
1848,-4851.178036,31843.903550,135.981716,-0.402077,-1.514024,0.574130,2.623213,1
2216,-9289.780568,17885.621302,50.241183,0.767994,-0.825964,1.740006,2.183811,0
1013,-1003.737302,88213.639053,101.410118,-0.775698,4.180367,-0.437124,4.803781,2
1241,-885.598953,76614.944379,144.274911,1.513740,1.336485,0.459290,3.088781,2


In [103]:
from sklearn.preprocessing import MinMaxScaler

In [114]:
mmscler = MinMaxScaler()
train_x_mm = mmscler.fit_transform(train.iloc[:, :-1].values)
test_x_mm = mmscler.fit_transform(test.iloc[:, :-1].values)

In [121]:
train_x, train_y = train_x_mm[:, :, np.newaxis], train.target.values
test_x, test_y = test_x_mm[:, :, np.newaxis], test.target.values
print(train_x.shape, test_x.shape, train_y.shape, test_y.shape)

(40912, 7, 1) (2920, 7, 1) (40912,) (2920,)


In [120]:
train_x_mm[np.newaxis].shape

(40912, 7, 1)

In [91]:
import tensorflow as tf
from tensorflow.keras.layers import RNN, Dense, LSTM
from tensorflow.keras.models import Sequential

In [7]:
tf.__version__

'2.0.0-rc0'

In [94]:
dir(tf.keras.layers)

['AbstractRNNCell',
 'Activation',
 'ActivityRegularization',
 'Add',
 'AdditiveAttention',
 'AlphaDropout',
 'Attention',
 'Average',
 'AveragePooling1D',
 'AveragePooling2D',
 'AveragePooling3D',
 'AvgPool1D',
 'AvgPool2D',
 'AvgPool3D',
 'BatchNormalization',
 'Bidirectional',
 'Concatenate',
 'Conv1D',
 'Conv2D',
 'Conv2DTranspose',
 'Conv3D',
 'Conv3DTranspose',
 'ConvLSTM2D',
 'Convolution1D',
 'Convolution2D',
 'Convolution2DTranspose',
 'Convolution3D',
 'Convolution3DTranspose',
 'Cropping1D',
 'Cropping2D',
 'Cropping3D',
 'Dense',
 'DenseFeatures',
 'DepthwiseConv2D',
 'Dot',
 'Dropout',
 'ELU',
 'Embedding',
 'Flatten',
 'GRU',
 'GRUCell',
 'GaussianDropout',
 'GaussianNoise',
 'GlobalAveragePooling1D',
 'GlobalAveragePooling2D',
 'GlobalAveragePooling3D',
 'GlobalAvgPool1D',
 'GlobalAvgPool2D',
 'GlobalAvgPool3D',
 'GlobalMaxPool1D',
 'GlobalMaxPool2D',
 'GlobalMaxPool3D',
 'GlobalMaxPooling1D',
 'GlobalMaxPooling2D',
 'GlobalMaxPooling3D',
 'Input',
 'InputLayer',
 'Input

In [159]:
model = Sequential()
model.add(SimpleRNN(8, input_shape=(7, 1)))
model.add(Dense(4, activation='relu'))
model.add(Dense(1, activation='linear'))
model.compile(optimizer='rmsprop', loss='mse')
model.summary()

Model: "sequential_50"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_47 (SimpleRNN)    (None, 8)                 80        
_________________________________________________________________
dense_53 (Dense)             (None, 4)                 36        
_________________________________________________________________
dense_54 (Dense)             (None, 1)                 5         
Total params: 121
Trainable params: 121
Non-trainable params: 0
_________________________________________________________________


In [160]:
model.fit(train_x, train_y, 16, epochs=15)

Train on 40912 samples
Epoch 1/15
40912/40912 [==============================] - 16s 397us/sample - loss: 1.5145
Epoch 2/15
24480/40912 [================>.............] - ETA: 5s - loss: 1.4007

KeyboardInterrupt: 

In [79]:
model.predict(test_x)

array([[1.0724037 , 0.88629264, 0.73052096, ..., 1.2717209 , 1.2005309 ,
        1.3559011 ],
       [1.0724037 , 0.88629264, 0.73052096, ..., 1.2717209 , 1.2005309 ,
        1.3559011 ],
       [1.0724037 , 0.88629264, 0.73052096, ..., 1.2717209 , 1.2005309 ,
        1.3559011 ],
       ...,
       [1.5328538 , 1.379627  , 1.249762  , ..., 1.4934411 , 1.4333178 ,
        1.6462848 ],
       [1.0724037 , 0.88629264, 0.73052096, ..., 1.2717209 , 1.2005309 ,
        1.3559011 ],
       [2.314567  , 2.2895498 , 2.265068  , ..., 2.4645321 , 2.3857918 ,
        2.397686  ]], dtype=float32)

In [158]:
model.evaluate(test_x, test_y)

2920/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

1.153241393745762